In [1]:
import json
import os
import sys
from math import *
wd = os.path.abspath(os.getcwd()) # os.path.dirname(os.path.dirname(os.getcwd()))
wd = os.path.dirname(wd)
sys.path.append(wd+"\\src")
import numpy as np


In [2]:
from gaslayer import GasFluxCalculator, GasEOS, GasLayer,GridStrecher,Powder,PowderOvLayer

In [3]:
from tube import Tube

In [4]:
with open('gpowders.json') as f:
    all_powders = json.load(f)

In [5]:
all_powders[r'4\7']

{'name': '4\\7',
 'f': 1.027,
 'etta': 0.228,
 'alpha_k': 1.008,
 'T_1': 3006.0,
 'ro': 1.6,
 'I_k': 0.32,
 'Z_k': 1.488,
 'k_1': 0.811,
 'lambda_1': 0.081,
 'k_2': 0.505,
 'lambda_2': -1.024,
 'k_f': 0.0003,
 'k_l': 0.0016}

In [6]:
powder_eos = Powder.from_bd(all_powders[r'4\7'])

In [7]:
flux_calculator = GasFluxCalculator()
grid_strecher = GridStrecher()

In [23]:
d = 23 * 1e-3
tube = Tube([0,1], [d, d])

In [24]:
omega = 30 * 1e-3
q = 170 *1e-3
delta = 700
p_ign = 5e6
pf = 300E6
W_0 = omega/delta
x0 = 0
x1 = tube.get_x2(x0,W_0)
x1

0.10315200200386698

In [54]:
n_cells = 100
layer = PowderOvLayer(n_cells=n_cells, tube=tube, powder=powder_eos, flux_calculator=flux_calculator, grid_strecher=grid_strecher)
layer.xs_borders = np.linspace(x0, x1, n_cells+1)
def foo(x, *args):
    return delta, p_ign, 0, 0
layer.init_ropue_fromfoo(foo)
layer.init_taus_acustic()

In [55]:
def get_V(tau: float, lr: PowderOvLayer):
    v_right = lr.Vs_borders[-1]
    p_right = lr.ps[-1]
    x_right = lr.xs_borders[-1]
    if p_right > pf or abs(v_right) > 1e-3:
        a = lr.S[-1] * p_right / q
        return v_right + tau * a
    return v_right

In [56]:
courant = 0.5
layers_all = []
while layer.xs_borders[-1] < 1:
    tau = layer.get_tau_min() * courant
    vr = get_V(tau, layer)
    layer1 = layer.step_simple(tau, 0, vr)
    if layer == layer1:
        break
    layer = layer1

In [57]:
d = layer.to_dict()

In [58]:
print(d['us'])

[  4.02397602  12.06935764  20.11667813  28.16217285  36.20829901
  44.25800366  52.29979255  60.34474662  68.39365307  76.43392849
  84.4758516   92.51813639 100.56142697 108.60662807 116.64370251
 124.6811408  132.7189513  140.7579057  148.7974081  156.82938569
 164.85859034 172.88499529 180.91418845 188.95040806 196.98381856
 205.00929945 213.03267782 221.05529311 229.07717497 237.09813236
 245.11893886 253.13734792 261.15078076 269.16272971 277.17410888
 285.18508404 293.19298889 301.19692055 309.20260175 317.20987358
 325.2145361  333.21691343 341.21789757 349.21860039 357.21795045
 365.21272872 373.20261238 381.19075937 389.17708399 397.16127292
 405.14188041 413.11690431 421.08528132 429.04833874 437.00548519
 444.95609404 452.89567428 460.82222993 468.73816999 476.64297069
 484.53488095 492.40893017 500.26522759 508.10745875 515.93412333
 523.74057118 531.52522857 539.28837315 547.03226112 554.75644194
 562.45746649 570.1377369  577.79814571 585.43839042 593.05798121
 600.65867